In [90]:
import os, sys
sys.path.insert(0, '/home/jtorrenc/mvesc/ETL')
from mvesc_utility_functions import *
import numpy as np
import pandas as pd
import random
from functools import partial

In [116]:
def color_gt(val,x=.5,c='blue'):
    """
    Colors values greater than x blue (or other provided color)
    
    :param float x: lower bound on colored values
    :param str c: color
    :returns: css string to color values in dataframe
    :rtype: str
    """
    color = c if val > x else 'black'
    return 'color: %s' % color

def table_exists(cursor, table, schema='clean'):
    """
    Checks to see if a table exists in the database
    
    :param pg.cursor cursor: 
    :param str table:
    :param str schema:
    :rtype: bool
    """
    cursor.execute("""                                                   
            select count(*) from information_schema.tables                       
            where table_schema = %s and table_name = %s
            """, [schema, table])
    return cursor.fetchall()[0][0]

In [129]:
with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        cursor.execute('select distinct model_name from model.reports')
        model_list = [m[0] for m in cursor.fetchall()]
        best = dict()
        for m in model_list:
            cursor.execute("""select filename from model.reports where model_name = %s
                            order by val_precision_10 desc limit 1; """, [m])
            best[m] = cursor.fetchall()[0][0]
    predictions = pd.DataFrame()
    for m, f in best.items():
        df = read_table_to_df(connection, f,columns = ['student_lookup','predicted_score'],
                                          schema='predictions')
        df.set_index('student_lookup',inplace=True)
        df.columns = [m]
        predictions = pd.concat((predictions,df), axis=1, join='inner')


In [120]:
kendall = predictions.corr('kendall')
spearman = predictions.corr('spearman')

# Kendall's Tau Prediction Correlation

In [121]:
kendall.style.applymap(partial(color_gt,x=.65,c='blue'))

# Spearman's Prediction Correlation

In [122]:
spearman.style.applymap(partial(color_gt,x=.75,c='blue'))

In [207]:
with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        cursor.execute('select distinct model_name from model.reports')
        model_list = [m[0] for m in cursor.fetchall()]
        cursor.execute('select distinct feature_categories from model.reports')
        feature_list = [f[0] for f in cursor.fetchall()]
        to_remove = []
        for f in feature_list:
            if ',' in f:
                to_remove.append(f)
        for f in to_remove:
            feature_list.remove(f)
        best = dict()
        features = dict()
        spearman = dict()
        for f in feature_list:
            best[f] = dict()
            for m in model_list:
                cursor.execute("""select filename from model.reports 
                                where model_name = %s
                                    and feature_categories = %s
                                    and feature_grades like '5, 6, 7, 8, 9'
                                order by val_precision_10 desc limit 1; """, [m,f])
                best[f][m] = cursor.fetchall()[0][0]
            temp = pd.DataFrame()
            for m, file in best[f].items():
                if table_exists(cursor, file, 'feature_scores'):
                    df = read_table_to_df(connection, file ,columns=['feature','importance'],schema='feature_scores')
                    df.set_index('feature',inplace=True)
                    df.columns = [m]
                    temp = pd.concat((temp,df), axis=1, join='inner')
            features[f] = temp
            spearman[f] = temp.corr('spearman')

# Spearman's Feature Correlation


In [208]:
features['absence'].sort_values('GB')

,GB,AB,RF,DT,SVM,SGD,ET,logit
feature,,,,,,,,
absence_consec_gr_7,0.000000,0.000000,0.000789,0.000000,0.087852,22.639499,0.000924,0.262957
tardy_gr_7,0.000000,0.000000,0.000713,0.000000,0.036175,9.322147,0.001442,0.000000
tardy_gr_8,0.006308,0.000000,0.018110,0.000000,-0.250450,-23.971235,0.023138,-0.189939
tardy_unexcused_gr_8,0.007643,0.000000,0.017005,0.000000,0.250397,2.663471,0.021831,0.057135
medical_gr_8,0.008812,0.004190,0.010744,0.000000,-0.000018,-17.312558,0.019220,0.097306
medical_gr_9,0.009512,0.034787,0.017815,0.059066,0.000043,-10.653882,0.021578,-0.010598
absence_consec_gr_8,0.011905,0.003216,0.028656,0.000000,-0.000016,1.331735,0.028532,0.031852
tardy_unexcused_gr_9,0.020969,0.030312,0.032665,0.000000,-0.000182,-6.658676,0.028906,0.072499
absence_unexcused_gr_7,0.039278,0.000000,0.033406,0.000000,-0.000055,-0.665868,0.042930,0.000000


In [209]:
spearman['absence'].style.applymap(partial(color_gt,x=.8,c='blue'))